In [328]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score
#from IPython.display import display, clear_output
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score as sk_f1_score

In [329]:
data = pd.read_csv('data/chips.txt', header=None, names=['X', 'Y', 'Class'])
data = data.sample(frac=1, random_state=0)
xs = data[['X', 'Y']].values
ys = data['Class'].values

#arr = np.genfromtxt('data/chips.txt', delimiter=',')
#xs, ys = arr[:,:2].copy(), np.int32(arr[:, 2:].ravel())
xs = StandardScaler().fit_transform(xs)
# ys = 2 * ys - 1

In [330]:
def f1_score(predicted, true):
    tp = ((predicted == true) * (predicted == 1)).sum()
    tn = ((predicted == true) * (predicted != 1)).sum()
    fp = ((predicted != true) * (predicted == 1)).sum()
    fn = ((predicted != true) * (predicted != 1)).sum()
    p = tp*1.0/max(1, tp+fp)
    r = tp*1.0/max(1, tp+fn)
    return (2*p*r/max(1, p+r), tp, fn, fp, tn)

In [331]:
def print_f1Nmatrix(t):
    print("confusion matrix")
    print("    T   F")
    print("P ", t[1], t[3])
    print("N ", t[4], t[2])
    print("f1_score = ", t[0])

In [332]:
def shuffle(X, y):
    p = np.arange(0, X.shape[0])
    np.random.shuffle(p)
    return (X[p], y[p])
np.random.seed(42)
#xs, ys = shuffle(xs, ys)

In [333]:
#plot1 = data.plot(kind='scatter', x='X', y='Y', c='Class', grid=True)
#plot1.set_facecolor("gray")

In [334]:
def split(X, y, part=10):
    p = X.shape[0] // part
    return X[p:], y[p:], X[:p], y[:p]

In [335]:
def space_function(x):
    return np.array([x[0], x[1], x[0]**2+x[1]**2, x[0]**2+2*x[0]*x[1]+x[1]**2])
    #return np.array([x[0], x[1], (x[0]**2 + x[1]**2) ** 0.5, np.arctan2(x[0], x[1])])


In [336]:
from sklearn.cross_validation import KFold
def kf_cross_validation(regressor, xs, ys, n_fold=10, kf=None, **params):
    measure = lambda x, y: f1_score(x, y)[0]
    kf_sum = 0
    f1_scores = []
    if kf is None:
        kf = KFold(len(xs), n_fold, True, 0)
    fold_num = 0
    P = 100
    for train_i, test_i in kf:
        regressor.fit(xs[train_i], ys[train_i], **params)
        predicted = regressor.predict(xs[test_i])
        meas = measure(predicted, ys[test_i])
        f1_scores.append(meas*P)
        kf_sum += meas
        fold_num += 1
        # print("Fold {} done, measure = {}".format(fold_num, meas))
    return (kf_sum/len(kf), np.array(f1_scores))

In [337]:
class SVM_SGD(BaseEstimator):
    def __init__(self, C, lr=0.01, eps=1e-5, iters=100, phi=None):
        self.phi = phi
        self.C = C
        self.lr = lr
        self.eps = eps
        self.iters = iters
        
    def get_params(self, deep=False):
        
        return {"C": self.C,
                "phi": self.phi,
                "lr": self.lr,
                "eps": self.eps,
                "iters": self.iters}

    def set_params(self, **params):
        self.C = params["C"]
        self.phi = params["phi"]
        self.lr = params["lr"]
        self.eps = params["eps"]
        self.iters = params["iters"]
        
    def fit(self, X, Y):
        #K(x1, x2)=<phi(x1), phi(x2)>        
        if self.phi is not None:
            X = np.apply_along_axis(self.phi, 1, X)
        teta = 1.0 / (2 * self.C)
        Y = 2 * Y - 1
        n = X.shape[0]
        dim = X.shape[1]        

        last_obj= -np.inf
        t = self.lr
        lr = 1.

        #solve minimization problem
        w = np.zeros(dim)
        w0 = 0
        for it in range(self.iters):
            for x,y in zip(X, Y):
                margin = y * (np.dot(x, w) - w0)
                sl = np.maximum(1 - margin, 0)
                w = w - self.lr * (-sl * y * x + 2 * teta * w)
                w0 = w0 - self.lr * (sl * y)
            margin = (np.dot(X, w) - w0) * Y
            cur_obj = np.maximum(1 - margin, 0).sum() + teta * np.dot(w, w)
            if abs(last_obj - cur_obj) < self.eps:                
                break
            last_obj = cur_obj

            lr = lr*(1+lr*t*it)**-1
        self.w, self.w0 = w, w0                
    
    def predict(self, x):
        if self.phi is not None:
            x = np.apply_along_axis(self.phi, 1, x)
                    
        return (np.ndarray.astype(np.sign(np.dot(x, self.w) - self.w0), np.int32) + 1) / 2

In [338]:
print("No kernel, C = 100")
svm = SVM_SGD(C = 100)
svm.fit(xs, ys)
pr = svm.predict(xs)
print_f1Nmatrix(f1_score(pr, ys))

No kernel, C = 100
confusion matrix
    T   F
P  17 21
N  39 41
f1_score =  0.262250453721


In [339]:
print("phi (x, y, x**2+y**2, 2xy)")
for C in [1, 2, 10, 100, 1000]:
    svm = SVM_SGD(C = C, phi = space_function )
    print("C =", C)    
    print("f1_score =", kf_cross_validation(svm, xs, ys)[0])        
    print("--------------------------------------------------")

phi (x, y, x**2+y**2, 2xy)
C = 1
f1_score = 0.795621616945
--------------------------------------------------
C = 2
f1_score = 0.811030472795
--------------------------------------------------
C = 10
f1_score = 0.811030472795
--------------------------------------------------
C = 100
f1_score = 0.816177531619
--------------------------------------------------
C = 1000
f1_score = 0.816177531619
--------------------------------------------------


In [340]:
best_c = 10
best_f1_sc = 0
for ic in range(5, 15):
    svm = SVM_SGD(C = ic, phi = space_function )
    f1_sc_t = kf_cross_validation(svm, xs, ys)[0];
    if (f1_sc_t > best_f1_sc):
        best_c = ic
        best_f1_sc = f1_sc_t

In [341]:
print("Full dataset, C = ", best_c)
svm = SVM_SGD(C = best_c, phi = space_function)
svm.fit(xs, ys)
prediction_svm = svm.predict(xs)
print_f1Nmatrix(f1_score(prediction_svm, ys))
print("--------------------------------------------------")

Full dataset, C =  5
confusion matrix
    T   F
P  49 9
N  51 9
f1_score =  0.844827586207
--------------------------------------------------


In [354]:
def wilcoxon(x, y):
    assert len(x) == len(y)
    d = x - y
    d = np.compress(np.not_equal(d, 0), d, axis=-1)
    count = len(d)  
    r = np.sort(abs(d))    
    nr = len(r)
    for i in range(0, nr):
        j = i
        while j < nr and r[i] == r[j]:
            j += 1
        r[i:j] = (j*(j-1)/2 - i*(i-1)/2) / (j - i)    
    W = sum(r * np.sign(d))
    z_score = W / np.sqrt(nr*(nr+1)*(2*nr+1)/6)
    return W, z_score


In [361]:
import knn
n_fold = 7
kf = KFold(len(xs), n_fold, True, 7)

knn = knn.KNN()
svm = SVM_SGD(C = best_c, phi = space_function)

f1_knn, f1_scoresKnn = kf_cross_validation(knn, xs, ys, kf=kf, k=3, weight_f='uniform', metric_f='l2')
print("KNN f1_score:", f1_scoresKnn)
print("KNN f1_score:", f1_knn)
f1_svm, f1_scoresSvm = kf_cross_validation(svm, xs, ys, kf=kf)
print("KNN f1_score:", f1_scoresSvm)
print("SVM f1_score:", f1_svm)

KNN f1_score: [ 73.68421053  87.5         80.          73.68421053  54.54545455  70.
  66.66666667]
KNN f1_score: 0.722972203235
KNN f1_score: [ 73.68421053  87.5         93.33333333  84.21052632  66.66666667
  77.77777778  90.        ]
SVM f1_score: 0.818817878028


In [362]:
print("Wilcoxon")
wilcoxon(f1_scoresSvm, f1_scoresKnn)

Wilcoxon


(10.0, 1.3483997249264841)

In [363]:
pValueTable =[
[ 0, 0.995,0.99,0.975,   0.95, 0.9,  0.1,  0.05,0.025,0.01,0.005],
[ 1,  0,0 ,0.001, 0.004 ,0.016,2.706, 3.841, 5.024, 6.635, 7.879],
[ 2, 0.01 , 0.02 ,0.051,0.103,0.211,4.605 ,5.991  ,7.378  ,9.21   ,10.597],
[ 3, 0.072, 0.115,0.216,0.352,0.584,6.251 ,7.815  ,9.348  ,11.345 ,12.838],
[ 4, 0.207, 0.297,0.484,0.711,1.064,7.779 ,9.488  ,11.143 ,13.277 ,14.86],
[ 5, 0.412, 0.554,0.831,1.145,1.61 ,9.236 ,11.07  ,12.833 ,15.086 ,16.75],
[ 6, 0.676, 0.872,1.237,1.635,2.204,10.645, 12.592, 14.449, 16.812, 18.548],
[ 7, 0.989, 1.239,1.69 ,2.167,2.833,12.017, 14.067, 16.013, 18.475, 20.278],
[ 8, 1.344, 1.646,2.18 ,2.733,3.49 ,13.362, 15.507, 17.535, 20.09 , 21.955],
[ 9, 1.735, 2.088,2.7,  3.325,4.168,14.684, 16.919, 19.023, 21.666, 23.589],
[ 10, 2.156,2.558,3.247,3.94 ,4.865,15.989, 18.307, 20.483, 23.209, 25.188],
[ 11, 2.603,3.053,3.816,4.575,5.578,17.275, 19.675, 21.92 ,24.725 ,26.757]]

def chiSquare(exp, obs):
    x2 = ((obs-exp)**2/exp).sum()
    print("x^2 =", x2)
    
    r = len(exp) - 1
    j = 1
    while (j < len(pValueTable[r]) - 1 and x2 > pValueTable[r][j]):
        j += 1
    return pValueTable[0][j]

print("pValue")
print(chiSquare(f1_scoresKnn, f1_scoresSvm))

pValue
x^2 = 15.4504485119
0.01
